In [ ]:
#imports
import cupy as cp
import cupyx.scipy.sparse
import cudf
import numpy as np

In [14]:
#Getting out DataSet
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz
!gunzip -k reviews_Musical_Instruments_5.json.gz

--2022-01-28 17:01:42--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2460495 (2.3M) [application/x-gzip]
Saving to: ‘reviews_Musical_Instruments_5.json.gz’

reviews_Musical_Ins 100%[===================>]   2.35M  1.77MB/s    in 1.3s    

2022-01-28 17:01:44 (1.77 MB/s) - ‘reviews_Musical_Instruments_5.json.gz’ saved [2460495/2460495]



In [22]:
#reading out DataSet
ratings = cudf.read_json("reviews_Musical_Instruments_5.json", lines=True)

In [23]:
ratings.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...",[0,"Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,[13,The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""",[1,The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""",[0,Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,[0,This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [25]:
#Getting the count of ratings
count_ratings = len(ratings)
count_ratings

10261

The unixReviewTime column enables us to user the last review left by the user for validation. We will start by finding the largest (newest) review time and assigning it a marker, valid. 

However, the unixReviewTime only goes down to the day, so if a user does all of their reviews on the same day, then all of their reviews would end up in the validation set. 

We can add a tie-breaker, or timeBreaker, by adding a scaled version of the row index to the time.

In [26]:
scaled_index = ratings.index / count_ratings
ratings["timeBreaker"] = scaled_index + ratings["unixReviewTime"]

The as_index parameter for groupby will create a fresh numerical index as opposed to using the reviewerID as the index.

In [27]:
valid_ratings = ratings[["reviewerID", "timeBreaker"]].groupby(['reviewerID'], as_index=False).max()
valid_ratings["valid"] = True

valid_ratings.head()

,reviewerID,timeBreaker,valid
0,A2U7DXDNYBBLQJ,1.322698e+09,True
1,ASNGOBUI6WHNV,1.403050e+09,True
2,A3KF2K1ZMAZSX4,1.397779e+09,True
3,A27PF2GVKMJSAA,1.367885e+09,True
4,A1EFMEZJESPF76,1.390003e+09,True


Next, we merge the validation marker back onto our full ratings dataset to make it easier to pull the training data.

In [28]:
data_split = ratings.merge(valid_ratings, how="left", on=['reviewerID', "timeBreaker"])

data_split.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,timeBreaker,valid
0,A3P2J1W1RBUX6Q,B000EEJF4O,Tommy B,[0,This stuff is great. Leaves board clean and fa...,5.0,Try It,1283126400,"08 30, 2010",1.283126e+09,<NA>
1,A21N9ACLVOMDLP,B000EEJJ5Y,Adam G,[0,Just bought this to see if it would inspire an...,5.0,LOVE IT!,1315094400,"09 4, 2011",1.315094e+09,<NA>
2,A14VAT5EAX3D9S,B000EEJJ5Y,Jake,[0,This capo is great for adding another level to...,5.0,Jake,1363392000,"03 16, 2013",1.363392e+09,<NA>
3,A24AQ24CD6865K,B000EEJJ5Y,"Rick from Westport, MA",[0,I play in different tunings and just slapping ...,5.0,Lots of fun,1399334400,"05 6, 2014",1.399334e+09,<NA>
4,AOINAOO0NQRGN,B000EEJJ5Y,Sessue,[0,This partial capo from Kyser has the same desi...,5.0,Same quality as the bestselling Kyser full capo,1329782400,"02 21, 2012",1.329782e+09,<NA>


In [29]:
# To keep things clean, we'll replace the nulls in the valid column with False.
data_split['valid'].fillna(False, inplace = True)

In [30]:
# We'll keep a simplified table of the user-item validation ratings to make it easier to check our work later.
clean_columns = ["reviewerID", "asin", "overall"]
train_overall = data_split.loc[~data_split['valid']][clean_columns]

train_overall.head()

,reviewerID,asin,overall
0,A3P2J1W1RBUX6Q,B000EEJF4O,5.0
1,A21N9ACLVOMDLP,B000EEJJ5Y,5.0
2,A14VAT5EAX3D9S,B000EEJJ5Y,5.0
3,A24AQ24CD6865K,B000EEJJ5Y,5.0
4,AOINAOO0NQRGN,B000EEJJ5Y,5.0


In [39]:
valid_overall = data_split.loc[data_split['valid']][clean_columns]

valid_overall.head()

,reviewerID,asin,overall
1472,A2HDEN9SPBWC5I,B001R2LQWQ,5.0
1488,A95J3RNEFN425,B000NGVQKO,5.0
1504,AQOQUWM58SGFR,B0002E1G5C,5.0
1536,A20SQURQ3G1BGY,B000B6DHAS,5.0
1537,A2UGCWC46V7PPG,B000B6DHB2,3.0


In [40]:
# While we're at it, let's save some of this work for future notebooks.
save_columns = clean_columns + ['valid']

data_split[save_columns].to_csv('ratings.csv', index=False)

In [41]:
# let's load our ratings
ratings = cudf.read_csv("ratings.csv", lines=True)

In [73]:
ratings

,reviewerID,asin,overall,valid
0,A3P2J1W1RBUX6Q,B000EEJF4O,5.0,False
1,A21N9ACLVOMDLP,B000EEJJ5Y,5.0,False
2,A14VAT5EAX3D9S,B000EEJJ5Y,5.0,False
3,A24AQ24CD6865K,B000EEJJ5Y,5.0,False
4,AOINAOO0NQRGN,B000EEJJ5Y,5.0,False
...,...,...,...,...
10256,A7IZNVL7JXLM,B007T8OGLK,3.0,False
10257,A3EXWV8FNSSFL6,B007T8OGLK,4.0,False
10258,A3SUC2DJYHUU9Y,B007T8OGLK,3.0,False
10259,A26U7I13QO5E0C,B007T8OGLK,5.0,False


In [51]:
#Before we split into train and test, let's factorize our users and items. Our test set won't be effective if it doesn't use the same indexes as the training dataset.
user_indexes, user_mapping = ratings["reviewerID"].factorize()
item_indexes, item_mapping = ratings["asin"].factorize()

In [75]:
#While we're building our sparse matrix, let's get a sense of our data along the way. First, let's see how many users there are.
user_count = len(user_mapping)
user_count

1429

In [76]:
#let's see how many users there are.
item_count = len(item_mapping)
item_count

900

In [74]:
# Training
train_indexes = ~ratings["valid"]
valid_indexes = ratings["valid"]
overall = ratings["overall"]

def get_dataset(data_selector, user_indexes, item_indexes, overall):
    # als rows and columns based on indexes
    row = cp.asarray(user_indexes)
    column = cp.asarray(item_indexes)
    data = cp.asarray(overall)
    #sparse matrix
    sparse_data = cupyx.scipy.sparse.coo_matrix((data, (row, column)))
    mask = cp.asarray([1 for _ in range(len(data))], dtype=np.float32)
    #sparse_mask
    sparse_mask = cupyx.scipy.sparse.coo_matrix((mask, (row, column)))
    return row, column, data, sparse_data, sparse_mask

train_row, train_column, train_data, train_sparse, train_mask = get_dataset(
    train_indexes, user_indexes, item_indexes, overall)

valid_row, valid_column, valid_data, valid_sparse, valid_mask = get_dataset(
    valid_indexes, user_indexes, item_indexes, overall)

For instance, let's set our embeddings to 2. This means for each user, we're going to find a way to represent them using 2 numbers. We'll initialize our embedding with random values for now, and convert their range to be -1 to 1.

In [67]:
#Similarly, we'll find a way to represent each item with 2 numbers.
embeddings = 2
shape = (user_count, item_count)

def initalize_features(length, embeddings):
    return cp.random.rand(embeddings, length) * 2 - 1

user_features = initalize_features(shape[0], embeddings)
item_features = initalize_features(shape[1], embeddings)

In [69]:
# From the ALS math we can make this
def als(values, mask, features, scale=0.01):
    numerator = values.dot(features.T)
    squared_features = (features ** 2).sum(axis=0)
    denominator = scale + mask.dot(squared_features)

    return (numerator / denominator[:, None]).T

here is a function to test how well our model is learning. 
Calculate the Root Mean Square Error (RMSE) as our goal is to minimize the squared error.

In [68]:
def rmse(user_features, item_features, data, row, column):
    predictions = (user_features[:, row] * item_features[:, column]).sum(axis=0)
    mean_squared_error = ((predictions - data) ** 2).mean() ** 0.5
    
    return predictions, mean_squared_error

In [72]:
#If we repete the process inside this loop many times we will get a smaller RMSE when that learninng curve becomes so small we can say that's our final RMSE.
for _ in range(20):
    user_features = als(train_sparse, train_mask, item_features)
    item_features = als(train_sparse.T, train_mask.T, user_features)
    predictions, error = rmse(
        user_features, item_features, valid_data, valid_row, valid_column)

    print ("RMSE:", error)

RMSE: 0.7204554390259571
